In [1]:
import pandas as pd
import os

In [2]:
path_of_folder_containing_states_raw_data = "D:\loyalist\sem2\step2\states"
path_for_processed_state_files = os.path.join(path_of_folder_containing_states_raw_data,'output') #I am keeping it in same as above folder you can change this address.
combined_elections_file_path = "D:\\loyalist\\sem2\\step2\\combined_election_data.csv"
poll_data_538_file_path = "D:\\loyalist\\sem2\\step2\\poll_data.csv"

In [3]:
def camelCase(string):
    return string.title().replace(' ','')

In [4]:
import re

def extract_state_name_from_text(text):
    # Define a list of US state names (you can use a more comprehensive list if needed)
    state_names = [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
        "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
        "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
        "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
        "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
        "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina",
        "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
        "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas",
        "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin",
        "Wyoming", "DC"
    ]

    # Convert the state names to lowercase for case-insensitive matching
    state_names_lower = [state.lower() for state in state_names]

    # Use regular expressions to find a match
    pattern = r"\b(" + "|".join(state_names_lower) + r")\b"
    match = re.search(pattern, text.lower())

    # Return the matched state name (or None if no match is found)
    return match.group(1) if match else 'National'

In [5]:
poll_data = pd.read_csv(poll_data_538_file_path)

In [6]:
def extract_state_name(state):
    if ',' in state:
        state_name = state.split(', ')[1]
        if '2024' in state_name:
            return "National"
        else:
            return state_name
    else:
        return 'National'

def rectify_state_names(state):
    if 'NE' in state:
        return 'Nebraska'
    elif 'ME' in state:
        return 'Maien'
    else:
        return state

# Calculate mode of leader column for each state
def calculate_mode_leader(df):
    mode_leader = df.groupby('state')['leader'].apply(lambda x: x.mode().iloc[0])
    return mode_leader

# Calculate sum and average of lead column for each state
def calculate_lead_sum_avg(df):
    lead_sum = df.groupby('state')['lead'].sum()
    lead_avg = df.groupby('state')['lead'].mean()
    return lead_sum, lead_avg

# Replace "Even" with 0 in the 'lead' column
def replace_even_with_zero(lead):
    if 'even' in str(lead).lower():
        return 0
    else:
        return int(lead)

# Impute party affiliation based on leader name
def impute_party_affiliation(leader):
    if leader in ['Biden', 'Harris']:
        return 'Democrat'
    elif leader == 'Trump':
        return 'Republican'
    else:
        return 'Unknown'

In [7]:
# Filter out rows not containing values Biden, Harris, or Trump in leader row
poll_data = poll_data[poll_data['leader'].isin(['Biden', 'Harris', 'Trump'])]

# Impute party affiliation based on leader name
poll_data['leader'] = poll_data['leader'].apply(impute_party_affiliation)

In [8]:
poll_data['state']=poll_data['state'].apply(rectify_state_names)

In [9]:
poll_data['state'] = poll_data['state'].apply(extract_state_name)

In [10]:
poll_data['lead'] = poll_data['lead'].apply(replace_even_with_zero)

In [11]:
# Calculate mode of leader column for each state
mode_leader = calculate_mode_leader(poll_data)

In [12]:
# Calculate sum and average of lead column for each state
lead_sum, lead_avg = calculate_lead_sum_avg(poll_data)

In [13]:
poll_result_df = pd.DataFrame({
    'state': mode_leader.index,
    'leader': mode_leader.values,
    'lead_sum': lead_sum.values,
    'lead_avg': lead_avg.values
})

In [14]:
print("Resultant DataFrame:")
print(poll_result_df)

Resultant DataFrame:
             state      leader  lead_sum   lead_avg
0          Alabama  Republican        63  21.000000
1           Alaska  Republican       105  11.666667
2          Arizona  Republican       675   4.166667
3         Arkansas  Republican        18   9.000000
4       California    Democrat       773  20.891892
5         Colorado    Democrat        85   8.500000
6      Connecticut    Democrat        24  12.000000
7         Delaware    Democrat        41  13.666667
8          Florida  Republican       494   7.264706
9          Georgia  Republican       680   4.444444
10          Hawaii    Democrat        27  13.500000
11           Idaho  Republican        22  22.000000
12        Illinois    Democrat        19   9.500000
13         Indiana  Republican        94  13.428571
14            Iowa  Republican        72  10.285714
15          Kansas  Republican         2   2.000000
16        Kentucky  Republican        16  16.000000
17       Louisiana  Republican        44  1

In [15]:
processed_poll_data = {}
for index, row in poll_result_df.iterrows():
    state = camelCase(row['state'])
    processed_poll_data[state] = {
        'leader': row['leader'],
        'lead_sum': row['lead_sum'],
        'lead_avg': row['lead_avg']
    }

In [16]:
print(f"\nNumber of rows in result_df: {len(processed_poll_data)}")


Number of rows in result_df: 51


In [17]:
print(processed_poll_data)

{'Alabama': {'leader': 'Republican', 'lead_sum': 63, 'lead_avg': 21.0}, 'Alaska': {'leader': 'Republican', 'lead_sum': 105, 'lead_avg': 11.666666666666666}, 'Arizona': {'leader': 'Republican', 'lead_sum': 675, 'lead_avg': 4.166666666666667}, 'Arkansas': {'leader': 'Republican', 'lead_sum': 18, 'lead_avg': 9.0}, 'California': {'leader': 'Democrat', 'lead_sum': 773, 'lead_avg': 20.89189189189189}, 'Colorado': {'leader': 'Democrat', 'lead_sum': 85, 'lead_avg': 8.5}, 'Connecticut': {'leader': 'Democrat', 'lead_sum': 24, 'lead_avg': 12.0}, 'Delaware': {'leader': 'Democrat', 'lead_sum': 41, 'lead_avg': 13.666666666666666}, 'Florida': {'leader': 'Republican', 'lead_sum': 494, 'lead_avg': 7.264705882352941}, 'Georgia': {'leader': 'Republican', 'lead_sum': 680, 'lead_avg': 4.444444444444445}, 'Hawaii': {'leader': 'Democrat', 'lead_sum': 27, 'lead_avg': 13.5}, 'Idaho': {'leader': 'Republican', 'lead_sum': 22, 'lead_avg': 22.0}, 'Illinois': {'leader': 'Democrat', 'lead_sum': 19, 'lead_avg': 9.5},

In [18]:
#import pandas as pd

#file1 = pd.read_csv("D:\\loyalist\\sem2\\step2\\combined_election_results_with_years_Alabama_Wyoming(in).csv")
#file2 = pd.read_csv("D:\\loyalist\\sem2\\step2\\Wisconsin.csv")
#file3 = pd.read_csv("D:\\loyalist\\sem2\\step2\\Yugahang_congressional_election_results_corrected_with_years.csv")

#combined_data = pd.concat([file1, file2, file3])
#combined_data.to_csv('D:\\loyalist\\sem2\\step2\\combined_election_data.csv', index=False)

In [19]:
import pandas as pd
#combined_elections_file_path = "D:\\loyalist\\sem2\\step2\\combined_election_data.csv"
combined_elections = pd.read_csv(combined_elections_file_path)

In [20]:
combined_elections = combined_elections[combined_elections['party'].isin(['Democratic', 'Republican'])]

In [21]:
# Convert the 'votes' column to numeric values
combined_elections['votes'] = pd.to_numeric(combined_elections['votes'], errors='coerce')

In [22]:
Y_data = combined_elections.groupby(['state', 'party'])['votes'].sum().reset_index()

In [23]:
# Select the row with the highest number of votes for each state
Y_data = Y_data.loc[Y_data.groupby('state')['votes'].idxmax()]
print(Y_data)

                    state       party         votes
1                 Alabama  Republican  1.068050e+07
3                  Alaska  Republican  8.245490e+05
5                 Arizona  Republican  3.173323e+06
7                Arkansas  Republican  1.982605e+06
8              California  Democratic  1.773909e+08
11               Colorado  Republican  1.936647e+07
12            Connecticut  Democratic  8.898921e+06
15               Delaware  Republican  3.658891e+06
17   District of Columbia  Republican  3.811688e+07
19                Florida  Republican  3.811688e+07
21                Georgia  Republican  1.862591e+07
22                 Hawaii  Democratic  6.975977e+06
25                  Idaho  Republican  4.163424e+06
26               Illinois  Democratic  1.825487e+07
29                Indiana  Republican  1.245955e+07
31                   Iowa  Republican  5.589391e+06
33                 Kansas  Republican  6.860393e+06
35               Kentucky  Republican  1.763488e+07
37          

In [24]:
Y_dict_temp = Y_data.set_index(['state', 'party']).to_dict('index')
print(Y_dict_temp)

{('Alabama', 'Republican'): {'votes': 10680498.0}, ('Alaska', 'Republican'): {'votes': 824549.0}, ('Arizona', 'Republican'): {'votes': 3173323.0}, ('Arkansas', 'Republican'): {'votes': 1982605.0}, ('California', 'Democratic'): {'votes': 177390876.0}, ('Colorado', 'Republican'): {'votes': 19366471.0}, ('Connecticut', 'Democratic'): {'votes': 8898921.0}, ('Delaware', 'Republican'): {'votes': 3658891.0}, ('District of Columbia', 'Republican'): {'votes': 38116883.0}, ('Florida', 'Republican'): {'votes': 38116883.0}, ('Georgia', 'Republican'): {'votes': 18625912.693}, ('Hawaii', 'Democratic'): {'votes': 6975977.0}, ('Idaho', 'Republican'): {'votes': 4163424.0}, ('Illinois', 'Democratic'): {'votes': 18254873.0}, ('Indiana', 'Republican'): {'votes': 12459554.0}, ('Iowa', 'Republican'): {'votes': 5589391.0}, ('Kansas', 'Republican'): {'votes': 6860393.0}, ('Kentucky', 'Republican'): {'votes': 17634878.0}, ('Louisiana', 'Republican'): {'votes': 9755742.0}, ('Maine', 'Democratic'): {'votes': 212

In [25]:
Y_dict = {}
for key,value in Y_dict_temp.items():
    Y_dict[camelCase(key[0])]={
        'party':key[1],
        'votes':value['votes']
    }
print(Y_dict)

{'Alabama': {'party': 'Republican', 'votes': 10680498.0}, 'Alaska': {'party': 'Republican', 'votes': 824549.0}, 'Arizona': {'party': 'Republican', 'votes': 3173323.0}, 'Arkansas': {'party': 'Republican', 'votes': 1982605.0}, 'California': {'party': 'Democratic', 'votes': 177390876.0}, 'Colorado': {'party': 'Republican', 'votes': 19366471.0}, 'Connecticut': {'party': 'Democratic', 'votes': 8898921.0}, 'Delaware': {'party': 'Republican', 'votes': 3658891.0}, 'DistrictOfColumbia': {'party': 'Republican', 'votes': 38116883.0}, 'Florida': {'party': 'Republican', 'votes': 38116883.0}, 'Georgia': {'party': 'Republican', 'votes': 18625912.693}, 'Hawaii': {'party': 'Democratic', 'votes': 6975977.0}, 'Idaho': {'party': 'Republican', 'votes': 4163424.0}, 'Illinois': {'party': 'Democratic', 'votes': 18254873.0}, 'Indiana': {'party': 'Republican', 'votes': 12459554.0}, 'Iowa': {'party': 'Republican', 'votes': 5589391.0}, 'Kansas': {'party': 'Republican', 'votes': 6860393.0}, 'Kentucky': {'party': '

In [26]:
print(len(Y_dict))

51


In [27]:
import os
import pandas as pd

In [28]:
def process_folder(folder_path, output_dir):
    # Initialize an empty dictionary to store the dataframes
    dfs_dict = {}

    # Iterate over each file in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is a CSV file
        if filename.endswith(".csv"):
            # Construct the full file path
            input_file_path = os.path.join(folder_path, filename)

            df = pd.read_csv(input_file_path)
            state = camelCase(extract_state_name_from_text(filename))

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
    
            output_file_path = os.path.join(output_dir,state+'.csv')
            chunk_size = 11000
            result_df = []
            while len(result_df)<5000:
                chunk_size = max(1000,chunk_size-1000)
                print(f'using chunksize {chunk_size} for {state}')
                result_df = process_file(df, chunk_size, poll_data, Y_data, state)
                if chunk_size == 1000:
                    break
                
            result_df.to_csv(output_file_path, index=False)
            print(f"done {state} with chunk size {chunk_size}")

In [29]:
group_features = ['Gender', 'Race', 'Veteran Status', 'Disability Status', 'Employment Status',
                  'Industry Sector', 'Occupancy Status', 'Mortgage Status', 
                  'Total Household Income', 'Health Coverage', 'Highest Education Level']

In [30]:

def process_file(df, chunk_size, poll_data, Y_data, state ):
    result_df = pd.DataFrame()

    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        grouped_data = chunk.groupby(group_features).size().reset_index(name='count')
        result_df = pd.concat([result_df, grouped_data], ignore_index=True)
    
    result_df['State']=state
    result_df['Leading Party']=processed_poll_data[state]['leader']
    result_df['Lead Sum']=processed_poll_data[state]['lead_sum']
    result_df['Lead Avg']=processed_poll_data[state]['lead_avg']

    result_df['Target Party'] = Y_dict[state]['party']
    result_df['Target Votes'] = Y_dict[state]['votes']
    result_df.columns = [camelCase(col.upper()) for col in result_df.columns]
    return result_df

In [31]:
process_folder(path_of_folder_containing_states_raw_data, path_for_processed_state_files)

using chunksize 10000 for Hawaii
done Hawaii with chunk size 10000
using chunksize 10000 for Idaho
done Idaho with chunk size 10000
using chunksize 10000 for Illinois
done Illinois with chunk size 10000
using chunksize 10000 for Indiana
done Indiana with chunk size 10000
using chunksize 10000 for Iowa
done Iowa with chunk size 10000
using chunksize 10000 for Kansas
done Kansas with chunk size 10000
using chunksize 10000 for Kentucky
done Kentucky with chunk size 10000
using chunksize 10000 for Louisiana
done Louisiana with chunk size 10000
using chunksize 10000 for Maine
done Maine with chunk size 10000
using chunksize 10000 for Maryland
done Maryland with chunk size 10000


In [32]:
#combine csv files in a folder

def combine_csv_files(folder_path):
    dataframes = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder_path, filename))
            dataframes.append(df)
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Save the combined dataframe to a new CSV file
    combined_df.to_csv(os.path.join(folder_path,'combined_data.csv'), index=False)
        

In [33]:
combine_csv_files(path_for_processed_state_files)